In [44]:
import json
import pandas as pd
import cmath
import xml.etree.ElementTree as et
import numpy as np
from transformations import *
from datetime import datetime, timedelta


In [22]:
global keeperA
global keeperB
global teamA
global teamB
global current_possession,poss_types, withshot,withshotongoal, withgoal, flank

In [47]:
format_str = "%H:%M:%S.%f"
timestamp_str = '0:00:20.000' 
time_duration1 = datetime.strptime(timestamp_str, format_str)
time_duration2 = datetime.strptime('0:00:50.000', format_str)
time_duration > timedelta(seconds=14)

True

In [23]:
pd.set_option('display.max_columns', 5000)
#pd.set_option('display.max_rows', None)

In [24]:
#instatiate empty dataframe with wyscout format
template_path='template.csv'
df=pd.read_csv(template_path)
wyscoutdf=df

In [25]:
#set all unknown attributes
id=videoTimestamp=relatedEventId=teamId=opponentTeamId=playerId=passrecipientId=possessionId = 0
possessioneventIndex=possessionteamId=groundDuelopponentId=groundDuelrelatedDuelId=infractionopponentId=shotgoalkeeperActionId=shotgoalkeeperId=aerialDuelopponentId=aerialDuelrelatedDuelId = 0
passheight =groundduel_take_on= ground_side=None
postshotxg=shotxg=shotgoalzone=possessionattackxg=aerialduelheight=aerialFirsttouch=aerialdueloppheight=np.nan


In [53]:
#to do
carry_x=carry_y=carry_prog=possessioneventsNumber=np.nan

In [26]:
#load instat file
instat_path = "data/National_Men_2022-05-16_Youth Club Friendlies_Sweden U16_vs_Switzerland U16_2267591_2022-05-16_Sweden U16_vs_Switzerland U16_2267591_rawdata.xml"
instatdf=pd.read_xml(instat_path)
matchId=getmatchId(instatdf)
instatdf=instatdf.iloc[2:,:]
instatdf.drop(columns=['column','number','dl', 'uid', 'action_id', 'player_id', 'team_id', 'standart_id', 'ts', 'position_id', 'opponent_id', 'opponent_team_id','opponent_position_id', 'zone_id', 'zone_dest_id', 'possession_id', 'possession_team_id', 'possession_team_id', 'attack_status_id', 'attack_type_id', 'attack_team_id', 'body_id' ], inplace=True)
#get the goalkeepers
keeperA,keeperB= get_keepers(instatdf)



In [32]:
instatdf

,level_0,index,id,action_name,player_name,team_name,half,second,standart_name,position_name,opponent_name,opponent_team_name,opponent_position_name,zone_name,zone_dest_name,pos_x005F_x,pos_y,pos_dest_x005F_x,pos_dest_y,len,possession_name,possession_team_name,possession_time,possession_number,attack_status_name,attack_type_name,attack_flang_id,attack_flang_name,attack_team_name,attack_number,body_name,direction,gate_x005F_x,gate_y,shot_type,touches,shot_handling,assistant_id,assistant_name
0,0,22,15060.0,4-3-3,None,Sweden U16,1.0,0.00,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
1,1,34,15078.0,4-4-2 diamond,None,Switzerland U16,1.0,0.00,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2,2,54,15098.0,Attacking pass accurate,Mario Greco,Switzerland U16,1.0,0.00,Indirect free kick,Forward - Right central,Sami Bashaj,Switzerland U16,Defender - Right central,2C,2C,52.5,34.0,47.9,31.5,5.2,Start,Switzerland U16,0.0,1.0,Start,Positional attack,3.0,right flank,Switzerland U16,1.0,None,NaN,NaN,NaN,None,None,None,NaN,None
3,3,100,22392.0,Average position Full match,Alessandro Romano,Switzerland U16,1.0,0.00,None,None,None,None,None,None,None,39.2,32.5,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
4,4,101,22393.0,Average position Full match,Timo Weber,Switzerland U16,1.0,0.00,None,None,None,None,None,None,None,31.8,37.8,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,2725,2837,21131.0,Shot on target,Jayson Ezeb,Sweden U16,5.0,227.97,Penalty,None,Marcel Lapierre,Switzerland U16,None,4B,4B,94.0,34.0,105.0,34.1,11.0,End,Sweden U16,0.0,256.0,End,penalty attack,NaN,None,Sweden U16,183.0,Right foot,-1.0,-0.13,0.08,None,None,Controlled bounce,NaN,None
2726,2726,2838,21132.0,Supersaves,Marcel Lapierre,Switzerland U16,5.0,228.33,Open play,None,None,None,None,1B,None,0.6,34.6,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2727,2727,2839,21134.0,Goal,Alessandro Romano,Switzerland U16,5.0,272.33,Penalty,None,Liam Selin,Sweden U16,None,4B,4B,94.0,34.0,105.0,30.9,11.4,End,Switzerland U16,NaN,257.0,End,penalty attack,NaN,None,Switzerland U16,184.0,Left foot,1.0,3.11,2.00,None,None,None,NaN,None
2728,2728,2840,22864.0,Assisting,Alessandro Romano,Switzerland U16,5.0,272.33,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None


In [28]:
#remove unnecessairy rows
mask = (instatdf['second']==0.00) & (~instatdf['player_name'].isna()) & (instatdf['action_name']!='Attacking pass accurate')&(instatdf.index<100)
instatdf = instatdf[~mask]
todrop = ['line-up', 'Substitute player', '1st half','2nd half', 'Players, that created offside trap']
mask = ~instatdf['action_name'].isin(todrop)
instatdf = instatdf[mask]
teamA,teamB=getformations(instatdf)

In [31]:
instatdf=instatdf.reset_index()

In [80]:
instatdf.iloc[2820:]

,id,action_name,player_name,team_name,half,second,standart_name,position_name,opponent_name,opponent_team_name,opponent_position_name,zone_name,zone_dest_name,pos_x005F_x,pos_y,pos_dest_x005F_x,pos_dest_y,len,possession_name,possession_team_name,possession_time,possession_number,attack_status_name,attack_type_name,attack_flang_id,attack_flang_name,attack_team_name,attack_number,body_name,direction,gate_x005F_x,gate_y,shot_type,touches,shot_handling,assistant_id,assistant_name
2822,23052.0,Building-up окончание,None,Switzerland U16,2.0,3207.84,Open play,None,None,None,None,2L,None,30.3,48.5,NaN,NaN,NaN,Ball possession,Switzerland U16,NaN,249.0,None,Positional attack,1.0,left flank,Switzerland U16,176.0,None,NaN,NaN,NaN,None,None,None,NaN,None
2823,21118.0,Picking up,Simeon Jovanovski,Sweden U16,2.0,3209.53,Open play,Defender - Right central,None,None,None,3R,None,55.4,13.1,NaN,NaN,NaN,End,Switzerland U16,6.0,249.0,End,Positional attack,1.0,left flank,Switzerland U16,176.0,None,NaN,NaN,NaN,None,None,None,NaN,None
2824,21119.0,Half time,None,None,2.0,3210.95,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2825,21120.0,Penalty shootout start,None,None,5.0,0.00,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2826,21121.0,Goal,William Andersson,Sweden U16,5.0,0.00,Penalty,None,Marcel Lapierre,Switzerland U16,None,4B,4B,94.0,34.0,105.0,35.8,11.1,End,Sweden U16,NaN,250.0,End,penalty attack,NaN,None,Sweden U16,177.0,Right foot,-1.0,-1.81,2.15,None,None,None,NaN,None
2827,22861.0,Assisting,William Andersson,Sweden U16,5.0,0.00,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2828,21122.0,Goal,Kiswend Tologo,Switzerland U16,5.0,40.30,Penalty,None,Liam Selin,Sweden U16,None,4B,4B,94.0,34.0,105.0,34.9,11.0,End,Switzerland U16,NaN,251.0,End,penalty attack,NaN,None,Switzerland U16,178.0,Left foot,1.0,-0.85,1.83,None,None,None,NaN,None
2829,22862.0,Assisting,Kiswend Tologo,Switzerland U16,5.0,40.30,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2830,21123.0,Shot on target,Jimi Dos Reis Nikko,Sweden U16,5.0,72.32,Penalty,None,Marcel Lapierre,Switzerland U16,None,4B,4B,94.0,34.0,105.0,31.9,11.2,End,Sweden U16,0.0,252.0,End,penalty attack,NaN,None,Sweden U16,179.0,Left foot,-1.0,2.13,0.60,None,None,Uncontrolled bounce,NaN,None
2831,21124.0,Supersaves,Marcel Lapierre,Switzerland U16,5.0,72.74,Open play,None,None,None,None,1B,None,0.6,35.9,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None


In [35]:
mask = (instatdf['second']==0)&(instatdf['action_name']!='Attacking pass accurate')&(instatdf.index<200)
instatdf=instatdf[~mask]
#ready for wyscout transformation

In [36]:
instatdf

,level_0,index,id,action_name,player_name,team_name,half,second,standart_name,position_name,opponent_name,opponent_team_name,opponent_position_name,zone_name,zone_dest_name,pos_x005F_x,pos_y,pos_dest_x005F_x,pos_dest_y,len,possession_name,possession_team_name,possession_time,possession_number,attack_status_name,attack_type_name,attack_flang_id,attack_flang_name,attack_team_name,attack_number,body_name,direction,gate_x005F_x,gate_y,shot_type,touches,shot_handling,assistant_id,assistant_name
2,2,54,15098.0,Attacking pass accurate,Mario Greco,Switzerland U16,1.0,0.00,Indirect free kick,Forward - Right central,Sami Bashaj,Switzerland U16,Defender - Right central,2C,2C,52.5,34.0,47.9,31.5,5.2,Start,Switzerland U16,0.0,1.0,Start,Positional attack,3.0,right flank,Switzerland U16,1.0,None,NaN,NaN,NaN,None,None,None,NaN,None
94,94,191,15099.0,Attacking pass accurate,Sami Bashaj,Switzerland U16,1.0,1.22,Open play,Defender - Right central,Jason Parente,Switzerland U16,Attacking midfielder - Central,2C,3C,47.9,31.5,71.6,44.6,27.1,Ball possession,Switzerland U16,0.0,1.0,Attack,Positional attack,3.0,right flank,Switzerland U16,1.0,None,NaN,NaN,NaN,None,None,None,NaN,None
95,95,192,15100.0,Air challenge,Jason Parente,Switzerland U16,1.0,3.84,Open play,Attacking midfielder - Central,Oscar Steinke Branby,Sweden U16,Defensive midfielder - Central,3C,None,71.6,44.6,NaN,NaN,NaN,Ball possession,Switzerland U16,0.0,1.0,Attack,Positional attack,3.0,right flank,Switzerland U16,1.0,None,NaN,NaN,NaN,None,None,None,NaN,None
96,96,193,15101.0,Picking up,Demir Xhemalija,Switzerland U16,1.0,8.47,Open play,Defensive midfielder - Central,None,None,None,3L,None,64.3,48.9,NaN,NaN,NaN,Ball possession,Switzerland U16,0.0,1.0,Attack,Positional attack,3.0,right flank,Switzerland U16,1.0,None,NaN,NaN,NaN,None,None,None,NaN,None
97,97,194,15102.0,Attacking pass accurate,Demir Xhemalija,Switzerland U16,1.0,9.47,Open play,Defensive midfielder - Central,Diego Benito Poloni,Switzerland U16,Defender - Left central,3L,2C,62.6,50.2,47.4,43.8,16.5,Ball possession,Switzerland U16,0.0,1.0,Attack,Positional attack,3.0,right flank,Switzerland U16,1.0,None,NaN,NaN,NaN,None,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,2725,2837,21131.0,Shot on target,Jayson Ezeb,Sweden U16,5.0,227.97,Penalty,None,Marcel Lapierre,Switzerland U16,None,4B,4B,94.0,34.0,105.0,34.1,11.0,End,Sweden U16,0.0,256.0,End,penalty attack,NaN,None,Sweden U16,183.0,Right foot,-1.0,-0.13,0.08,None,None,Controlled bounce,NaN,None
2726,2726,2838,21132.0,Supersaves,Marcel Lapierre,Switzerland U16,5.0,228.33,Open play,None,None,None,None,1B,None,0.6,34.6,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2727,2727,2839,21134.0,Goal,Alessandro Romano,Switzerland U16,5.0,272.33,Penalty,None,Liam Selin,Sweden U16,None,4B,4B,94.0,34.0,105.0,30.9,11.4,End,Switzerland U16,NaN,257.0,End,penalty attack,NaN,None,Switzerland U16,184.0,Left foot,1.0,3.11,2.00,None,None,None,NaN,None
2728,2728,2840,22864.0,Assisting,Alessandro Romano,Switzerland U16,5.0,272.33,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None


In [38]:
instatdf.iloc[2620:]

,level_0,index,id,action_name,player_name,team_name,half,second,standart_name,position_name,opponent_name,opponent_team_name,opponent_position_name,zone_name,zone_dest_name,pos_x005F_x,pos_y,pos_dest_x005F_x,pos_dest_y,len,possession_name,possession_team_name,possession_time,possession_number,attack_status_name,attack_type_name,attack_flang_id,attack_flang_name,attack_team_name,attack_number,body_name,direction,gate_x005F_x,gate_y,shot_type,touches,shot_handling,assistant_id,assistant_name
2713,2713,2825,21120.0,Penalty shootout start,None,None,5.0,0.00,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2714,2714,2826,21121.0,Goal,William Andersson,Sweden U16,5.0,0.00,Penalty,None,Marcel Lapierre,Switzerland U16,None,4B,4B,94.0,34.0,105.0,35.8,11.1,End,Sweden U16,NaN,250.0,End,penalty attack,NaN,None,Sweden U16,177.0,Right foot,-1.0,-1.81,2.15,None,None,None,NaN,None
2715,2715,2827,22861.0,Assisting,William Andersson,Sweden U16,5.0,0.00,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2716,2716,2828,21122.0,Goal,Kiswend Tologo,Switzerland U16,5.0,40.30,Penalty,None,Liam Selin,Sweden U16,None,4B,4B,94.0,34.0,105.0,34.9,11.0,End,Switzerland U16,NaN,251.0,End,penalty attack,NaN,None,Switzerland U16,178.0,Left foot,1.0,-0.85,1.83,None,None,None,NaN,None
2717,2717,2829,22862.0,Assisting,Kiswend Tologo,Switzerland U16,5.0,40.30,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2718,2718,2830,21123.0,Shot on target,Jimi Dos Reis Nikko,Sweden U16,5.0,72.32,Penalty,None,Marcel Lapierre,Switzerland U16,None,4B,4B,94.0,34.0,105.0,31.9,11.2,End,Sweden U16,0.0,252.0,End,penalty attack,NaN,None,Sweden U16,179.0,Left foot,-1.0,2.13,0.60,None,None,Uncontrolled bounce,NaN,None
2719,2719,2831,21124.0,Supersaves,Marcel Lapierre,Switzerland U16,5.0,72.74,Open play,None,None,None,None,1B,None,0.6,35.9,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2720,2720,2832,21126.0,Goal,Dorian Derbaci,Switzerland U16,5.0,103.04,Penalty,None,Liam Selin,Sweden U16,None,4B,4B,94.0,34.0,105.0,36.9,11.4,End,Switzerland U16,NaN,253.0,End,penalty attack,NaN,None,Switzerland U16,180.0,Right foot,1.0,-2.91,0.25,None,None,None,NaN,None
2721,2721,2833,22863.0,Assisting,Dorian Derbaci,Switzerland U16,5.0,103.04,Open play,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,None,None,None,NaN,None
2722,2722,2834,21127.0,Shot into the bar/post,Nino Bursic,Sweden U16,5.0,144.55,Penalty,None,Marcel Lapierre,Switzerland U16,None,4B,4B,94.0,34.0,105.0,36.3,11.2,End,Sweden U16,0.0,254.0,End,penalty attack,NaN,None,Sweden U16,181.0,Right foot,-1.0,-2.29,2.46,None,None,None,NaN,None


In [11]:
a,b,c=get_event_type(instatdf,2607,teamA,teamB,'P')

2607
['loss', 'touch_in_box']
action: Assisting
before output:
2609
postmatch_penalty
['loss', 'touch_in_box']


In [12]:
a,b,c

(2609, 'postmatch_penalty', ['touch_in_box', 'loss'])

In [ ]:
row=instatdf.iloc[2468]
print(row)

In [21]:
#instatiate first possession
current_possession,poss_types, withshot,withshotongoal, withgoal, flank=setnewpossession(instatdf,0) #start x, y, end x, y, possession_duration, possessionstart index|possession types, attack variables


In [24]:
def create_event(instat, ind):
    global current_possession,poss_types, withshot,withshotongoal, withgoal, flank
    index_instat, typeprimary, typesecondary = get_event_type(instat, ind)
    action = instat['action_name'].iloc[ind]
    #setup every attribute
    period=get_period(instat,ind)
    minute, second, matchTimestamp = get_time(instat, ind, period)
    locx, locy = get_location(instat, ind)
    teamname = instat['team_name'].iloc[ind]
    teamformation = teamA[1] if (teamname==teamA[0]) else teamB[1]
    oppteamname = teamB[0] if (teamname==teamA[0]) else teamA[0]
    oppteamformation = teamB[1] if (teamname==teamA[0]) else teamA[1]
    playername = instat['player_name'].iloc[ind]  #have to change to player names
    playerposition = position_transform(instat['position_name'].iloc[ind], teamformation)
    passlength = instat['len'].iloc[ind]
    isnewposs,possindex = newposs(instat, ind, index_instat)
    possteamname = instat['possession_team_name'].iloc[ind]
    possteamformation = teamA[1] if (possteamname==teamA[0]) else teamB[1]
    possession = instat['possession_name'].iloc[ind]
    keptPoss=None if ('defensive_duel' in typesecondary) else (False if possession=='Transition of possession' else True)
    stopped_prog=None if ('offensive_duel' in typesecondary) else (True if possession=='Transition of possession' else False)
    keepercoord_x, keepercoord_y=get_goalkeeper_coordinates(instat, ind, index_instat, keeperA, keeperB)
    reflexsave=False

    #passing
    if not (pd.isna(instat["pos_dest_x005F_x"].iloc[ind])):
        passaccurate=isaccurate_pass(action)
        passangle=calculate_angle(instat["pos_x005F_x"].iloc[ind], instat["pos_y"].iloc[ind], instat["pos_dest_x005F_x"].iloc[ind], instat["pos_dest_y"].iloc[ind])
        passend_x, passend_y=get_dest_location(instat,ind)
        if(passaccurate):
            passrec, passrec_pos_instat = get_pass_recipient(instat, ind)
            passrec_pos = position_transform(passrec_pos_instat, teamformation) 
        else:passrec= passrec_pos_instat=passrec_pos=np.nan
    else:
        passaccurate=passangle=passend_x=passend_y=passrec=passrec_pos=np.nan
    
    #shot
    if (isshot(action)):
        goalkeeper = keeperA[0] if (teamname!=keeperA[1]) else keeperB[0]
        bodypart=bodypart_transform(instat['body_name'].iloc[ind])
        isgoal= True if 'goal' in typesecondary else False
        isontarget= True if (action=='Shot on target' or action == 'Goal') else False
    else:
        bodypart=isgoal=isontarget=goalkeeper=np.nan

    #infraction
    if(action=='infraction'):
        isyellow=True if ('yellow_card' in typesecondary) else False
        isred = True if ('red_card' in typesecondary) else False
        infraction_type = 'regular_foul' if action!='Deferred foul' else 'late_card_foul' #there are more types of fouls
        infractionopp = instat['opponent_name'].iloc[ind]
        infractionopp_pos = position_transform(instat['opponent_position_name'].iloc[ind], oppteamformation)
    else:
        isyellow=isred=infraction_type=infractionopp=infractionopp_pos=np.nan

    #duel
    if(typeprimary=='duel'):
        aerialopp=aerialopp_pos=groundopp=groundopp_pos =grounddueltype=np.nan 
        ground_stopped_prog=ground_recov_poss =ground_side=groundkeptposs=groundduel_progressed_with_ball=np.nan

        duelopp = instat['opponent_name'].iloc[ind]
        duelopp_pos = position_transform(instat['opponent_position_name'].iloc[ind], oppteamformation)

        #adjust loss tag
        if ('loss' in typesecondary):
            if(stopped_prog==True):
                typesecondary.remove('loss')

        if ('aerial_duel' in typesecondary):
            aerialopp=duelopp
            aerialopp_pos = duelopp_pos
            
        else:
            
            groundopp=duelopp
            groundopp_pos = duelopp_pos
            grounddueltype = 'dribble' if 'dribble' in typesecondary else ('defensive_duel' if 'defensive_duel' in typesecondary else 'offensive_duel')
            groundkeptposs = keptPoss
            groundduel_progressed_with_ball=groundkeptposs #not so clear what progressed with ball means exactly
            ground_stopped_prog=stopped_prog
            ground_recov_poss =ground_stopped_prog #not so clear
            
    else:
        aerialopp=aerialopp_pos=groundopp=groundopp_pos =grounddueltype=np.nan 
        ground_stopped_prog=ground_recov_poss =ground_side=groundkeptposs=groundduel_progressed_with_ball=np.nan

    #possession
    if (current_possession[5]>ind):
        poss_x=poss_y=endposs_x=endposs_y=possessionType=np.nan
    else:
        poss_x=current_possession[0]
        poss_y=current_possession[1]
        endposs_x=current_possession[2]
        endposs_y=current_possession[3]
        possessionType=poss_types
    
    if (isnewposs==True):
        current_possession,poss_types, withshot,withshotongoal, withgoal, flank = setnewpossession(instat, possindex+1) 

    new_event = {
        "id": id,
        "matchId": matchId,
        "matchPeriod":get_period(instat,ind),
        "minute":minute,
        "second":second,
        "matchTimestamp":matchTimestamp,
        "videoTimestamp":videoTimestamp,
        "relatedEventId":relatedEventId,
        "shot":np.nan,
        "groundDuel":np.nan,
        "aerialDuel":np.nan,
        "infraction":np.nan,
        "carry":np.nan,
        "type.primary":typeprimary,	
        "type.secondary":typesecondary,
        "location.x":locx,	
        "location.y":locy,	
        "team.id":teamId,	
        "team.name": teamname,	
        "team.formation":teamformation,	
        "opponentTeam.id":opponentTeamId,
        "opponentTeam.name":oppteamname,	
        "opponentTeam.formation":oppteamformation,	
        "player.id":playerId,	
        "player.name":playername,	
        "player.position":playerposition,	
        "pass.accurate":passaccurate,	
        "pass.angle":passangle,	
        "pass.height":passheight,	
        "pass.length":passlength,	
        "pass.recipient.id":passrecipientId,	
        "pass.recipient.name":passrec,	
        "pass.recipient.position":passrec_pos,
        "pass.endLocation.x":passend_x,	
        "pass.endLocation.y":passend_y,	
        "possession.id":possessionId,	
        "possession.duration":current_possession[4],	
        "possession.types":possessionType,	
        "possession.eventsNumber":possessioneventsNumber,	
        "possession.eventIndex":possessioneventIndex,	
        "possession.startLocation.x":poss_x,
        "possession.startLocation.y":poss_y,	
        "possession.endLocation.x":endposs_x,	
        "possession.endLocation.y":endposs_y,	
        "possession.team.id":possessionteamId,	
        "possession.team.name":possteamname,	
        "possession.team.formation":possteamformation,	
        "possession.attack":np.nan,	
        "pass":np.nan,
        "groundDuel.opponent.id":groundDuelopponentId,	
        "groundDuel.opponent.name":groundopp,	
        "groundDuel.opponent.position":groundopp_pos,	
        "groundDuel.duelType":grounddueltype,	
        "groundDuel.keptPossession":groundkeptposs,	
        "groundDuel.progressedWithBall":groundduel_progressed_with_ball,	
        "groundDuel.stoppedProgress":ground_stopped_prog,	
        "groundDuel.recoveredPossession":ground_recov_poss,	
        "groundDuel.takeOn":groundduel_take_on,	
        "groundDuel.side":ground_side,
        "groundDuel.relatedDuelId":groundDuelrelatedDuelId,	
        "infraction.yellowCard":isyellow,	
        "infraction.redCard":isred,	
        "infraction.type":infraction_type,	
        "infraction.opponent.id":infractionopponentId,	
        "infraction.opponent.name":infractionopp,	
        "infraction.opponent.position":infractionopp_pos,	
        "possession.attack.withShot":withshot,	
        "possession.attack.withShotOnGoal":withshotongoal,	
        "possession.attack.withGoal":withgoal,	
        "possession.attack.flank":flank,	
        "possession.attack.xg":possessionattackxg,	
        "carry.progression":carry_prog,	
        "carry.endLocation.x":carry_x,	
        "carry.endLocation.y":carry_y,	
        "shot.bodyPart":bodypart,	
        "shot.isGoal":isgoal,	
        "shot.onTarget":isontarget,	
        "shot.goalZone":shotgoalzone,	
        "shot.xg": shotxg,	
        "shot.postShotXg":postshotxg,	
        "shot.goalkeeperActionId":shotgoalkeeperActionId,	
        "shot.goalkeeper.id":shotgoalkeeperId,	
        "shot.goalkeeper.name":goalkeeper,	
        "possession":np.nan,	
        "aerialDuel.opponent.id":aerialDuelopponentId,	
        "aerialDuel.opponent.name":aerialopp,	
        "aerialDuel.opponent.position":aerialopp_pos,	
        "aerialDuel.opponent.height":aerialdueloppheight,	
        "aerialDuel.firstTouch":aerialFirsttouch,	
        "aerialDuel.height":aerialduelheight,	
        "aerialDuel.relatedDuelId":aerialDuelrelatedDuelId,	
        "shot.goalkeeper":np.nan,	
        "infraction.opponent":np.nan,	
        "location":np.nan
    }
    
    #check when second event has to be generated
    if (typeprimary=='duel'):
        new_event_2=create_second_duel_event(new_event, keptPoss, stopped_prog, current_possession,poss_types, withshot,withshotongoal, withgoal, flank, isnewposs,ind)
        return index_instat,[new_event, new_event_2]
    elif (action=='Shot on target' or action=='Goal'):
        new_event_2=create_second_shot_event(new_event, keeperA, keeperB, keepercoord_x, keepercoord_y, reflexsave)
        return index_instat,[new_event, new_event_2]
     
    return index_instat, [new_event]

In [23]:
index = 0   
while (True):
    index, newevents = create_event(instatdf, index)
    for event in newevents:
        wyscoutdf = pd.concat([wyscoutdf, pd.DataFrame([event])], ignore_index=True)
    if(instatdf['action_name'].iloc[index]=='Match end'):
        break

C:\Users\Rui\AppData\Local\Temp\ipykernel_38484\2402195945.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  wyscoutdf = pd.concat([wyscoutdf, pd.DataFrame([event])], ignore_index=True)


TypeError: create_second_shot_event() missing 1 required positional argument: 'relfexsave'

In [ ]:
instatdf